In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup 
import time
import datetime 
from pytz import timezone
from selenium import webdriver

In [2]:
tick = pd.read_csv('/Users/kim/Documents/Python/Press release project/Stock_lists_20200621.txt', encoding ='latin1', sep = '\t')

10 x genomics bloomberg info
https://www.bloomberg.com/profile/company/1228021D:US

In [3]:
tick.head()

,Company,Ticker,Market
0,1+1 DRILLISCH AG O.N.,DRI,FRA
1,10x Genomics,TXG,NASDAQ
2,11 88 0 SOLUTIONS AG,TGT,FRA
3,2G ENERGY AG,2GB,FRA
4,3U HOLDING AG,UUU,FRA


In [6]:
for i in range(0, tick.Market.size):
    symbol = tick.Ticker[i]
    ## Replace the market by GR for FRA and US for NASDAQ and NYSE
    country = re.sub("FRA", "GR", tick.Market[i])
    country = re.sub("NASDAQ|NYSE", "US", country)
    # get the company info page from Bloomberg
    url = "https://www.bloomberg.com/profile/company/"+symbol+":"+country
    driver = webdriver.Firefox(executable_path=r"/Users/kim/documents/Python/Press release project/geckodriver.exe")
    driver.get(url)
    ## Wait for page laoding
    driver.implicitly_wait(1)
    try:
        company_feats = driver.find_elements_by_class_name("infoTableItemValue__e188b0cb")
        tick.loc[i, 'Sector'] = company_feats[0].text
        tick.loc[i, 'Industry'] = company_feats[1].text
        tick.loc[i, 'SubIndustry'] = company_feats[2].text
        tick.loc[i, 'Founded'] = company_feats[3].text
        tick.loc[i, 'Address'] = company_feats[4].text
        tick.loc[i, 'Phone'] = company_feats[5].text
        tick.loc[i, 'Website'] = company_feats[6].text
        tick.loc[i, 'NumEmployees'] = company_feats[7].text
    except:
        pass
    driver.quit()
tick.head()    

,Company,Ticker,Market,Sector,Industry,SubIndustry,Founded,Address,Phone,Website,NumEmployees
0,1+1 DRILLISCH AG O.N.,DRI,FRA,Communications,Telecommunications,Telecommunications,02/09/2004,"Wilhelm Roentgen Strasse 1-5 Maintal, 63477 Ge...",49-6181-412-143,www.drillisch.de,3191
1,10x Genomics,TXG,NASDAQ,Health Care,Health Care,Medical Equipment & Devices,07/02/2012,"6230 Stoneridge Mall Road Pleasanton, CA 94566...",1-925-401-7300,www.10xgenomics.com,713
2,11 88 0 SOLUTIONS AG,TGT,FRA,Technology,Software & Tech Services,Technology Services,08/01/2018,"Hohenzollernstrasse 24 Essen, 45128 Germany",49-201-8099-0,www.11880.com,545
3,2G ENERGY AG,2GB,FRA,Energy,Renewable Energy,Renewable Energy,01/25/2008,"Benzstrasse 3 Heek, D 48619 Germany",49-2568-9347-0,www.2-g.com,695
4,3U HOLDING AG,UUU,FRA,Industrials,Industrial Products,Electrical Equipment,07/17/2003,"Frauenbergstrasse 31₣ 1/64 5/1633 Marburg, 350...",49-6421-999-0,www.3u.net,205


In [9]:
tick.to_csv("/Users/kim/Documents/Python/Press release project/Stock_lists_20200621.txt",
            sep = "\t",
           index = False)